In [1]:
import datetime
import pandas as pd
import numpy as np

from pybaseball import schedule_and_record
print('ready')

ready


In [2]:
cubs = schedule_and_record(2018, 'CHC')

print('ready')

ready


In [4]:
cubs.head(15)

,Date,Tm,Home_Away,Opp,W/L,R,RA,Inn,W-L,Rank,GB,Win,Loss,Save,Time,D/N,Attendance,Streak,Orig. Scheduled
1,"Thursday, Mar 29",CHC,@,MIA,W,8.0,4.0,9.0,1-0,1.0,Tied,Cishek,Urena,None,3:18,D,32151.0,1.0,None
2,"Friday, Mar 30",CHC,@,MIA,L-wo,1.0,2.0,17.0,1-1,3.0,1.0,Despaigne,Butler,None,5:18,N,12034.0,-1.0,None
3,"Saturday, Mar 31",CHC,@,MIA,W,10.0,6.0,10.0,2-1,2.0,1.0,Strop,Ziegler,None,3:57,N,13422.0,1.0,None
4,"Sunday, Apr 1",CHC,@,MIA,L,0.0,6.0,9.0,2-2,3.0,1.5,Peters,Quintana,None,2:36,D,10428.0,-1.0,None
5,"Monday, Apr 2",CHC,@,CIN,L,0.0,1.0,9.0,2-3,4.0,2.5,Mahle,Chatwood,Iglesias,2:33,D,18963.0,-2.0,None
6,"Thursday, Apr 5",CHC,@,MIL,W,8.0,0.0,9.0,3-3,3.0,2.0,Lester,Suter,None,2:52,N,24310.0,1.0,None
7,"Friday, Apr 6",CHC,@,MIL,L-wo,4.0,5.0,9.0,3-4,3.0,3.0,Albers,Montgomery,None,3:45,N,37758.0,-1.0,None
8,"Saturday, Apr 7",CHC,@,MIL,W,5.0,2.0,9.0,4-4,3.0,2.0,Strop,Barnes,Morrow,3:26,D,43331.0,1.0,None
9,"Sunday, Apr 8",CHC,@,MIL,W,3.0,0.0,9.0,5-4,2.0,2.0,Quintana,Anderson,Morrow,2:41,D,39282.0,2.0,None
10,"Tuesday, Apr 10",CHC,Home,PIT,L,5.0,8.0,9.0,5-5,2.0,3.0,Nova,Chatwood,Vazquez,3:05,D,40144.0,-1.0,None


In [2]:
bballJoin = pd.read_csv('csv/bballJoin.csv', index_col=None)


In [3]:
print('Get Sox sched stuff')
sox = schedule_and_record(2018, 'CHW')
soxsort = pd.DataFrame ( sox.loc[ ( sox["W/L"].notnull() ) ] )
soxsort.sort_index(ascending=False,inplace=True)
soxlast = soxsort[:1]
soxnext = pd.DataFrame ( sox.loc[ ( sox["W/L"].isnull() ) ] )
soxnext = soxnext[:1]
soxlast = soxnext.append(soxlast)
soxlast = soxlast.rename(columns = {'Tm':'teamID'})
left = soxlast
right = bballJoin
soxnextlast = pd.merge(left, right, how='left', left_on='Opp', right_on='tres', suffixes=('_x', '_y'))
soxnextlast = soxnextlast.sort_values(by='R', ascending=False).reset_index(drop=True)
soxnextlast.to_csv("csv/soxnextlast.csv", index=False, encoding="utf-8")
print('Last and next sox games saved')
# Now aggregate results by team
# get df of teams played
soxteams = []
for name,grouped in soxsort.groupby(['Opp']):
    dlist = list([name])
    soxteams.append(dlist)
df = pd.DataFrame(data = soxteams, columns=['Team'])
# now aggregate results
soxlist = []
for team in df['Team']:
    # Link team in df to soxteams list in teamlist
    opp = soxsort['Opp'] == team
    df1 = soxsort[opp]
    # get wins and losses
    w = df1['W/L'] == 'W'
    wins = len( df1[w] )
    l = df1['W/L'] == 'L'
    loses = len( df1[l] )
    # average runs/against
    rAvg = np.round( df1['R'].mean() ,2)
    raAvg = np.round( df1['RA'].mean() ,2)
    # append to empty list
    slist = list([team,wins,loses,rAvg,raAvg])
    soxlist.append(slist)
# bring list into dataframe and name columns
soxagainst = pd.DataFrame(data = soxlist, columns=['teamID','Wins','Loses','AvgRuns','AvgRunsAg'])
# join aggregated list with team names
left = soxagainst
right = bballJoin
soxagg = pd.merge(left, right, how='left', left_on='teamID', right_on='tres', suffixes=('_x', '_y'))
soxagg.to_csv("csv/soxagg.csv", index=False, encoding="utf-8")
print("Sox aggregate done")

Get Sox sched stuff
Last and next sox games saved
Sox aggregate done


In [3]:
print('Get Sox sched stuff')
sox = schedule_and_record(2018, 'CHW')

Get Sox sched stuff


In [10]:
soxsort = pd.DataFrame ( sox.loc[ ( sox["W/L"].notnull() ) ] )
soxsort.sort_index(ascending=False,inplace=True)
soxlast = soxsort[:1]
soxlast = soxlast.copy()
soxlast.loc[:, 'R'] = soxlast['R'].astype(int)
soxlast.loc[:, 'R'] = soxlast['R'].astype(str)
soxlast.loc[:, 'RA'] = soxlast['RA'].astype(int)
soxlast.loc[:, 'RA'] = soxlast['RA'].astype(str)
soxlast.dtypes

Date                object
Tm                  object
Home_Away           object
Opp                 object
W/L                 object
R                   object
RA                  object
Inn                float64
W-L                 object
Rank               float64
GB                  object
Win                 object
Loss                object
Save                object
Time                object
D/N                 object
Attendance         float64
Streak             float64
Orig. Scheduled     object
dtype: object

In [11]:
soxlast.shape

(1, 19)

In [21]:
soxnext = pd.DataFrame ( sox.loc[ ( sox["W/L"].isnull() ) ] )
soxnext = soxnext[:1]
soxlast = soxnext.append(soxlast)
soxlast = soxlast.rename(columns = {'Tm':'teamID'})
left = soxlast
right = bballJoin
soxnextlast = pd.merge(left, right, how='left', left_on='Opp', right_on='tres', suffixes=('_x', '_y'))
soxnextlast = soxnextlast.sort_values(by='R', ascending=False).reset_index(drop=True)
soxnextlast.head()

,Date,teamID,Home_Away,Opp,W/L,R,RA,Inn,W-L,Rank,...,D/N,Attendance,Streak,Orig. Scheduled,Team,Tm,League,Division,color,tres
0,"Thursday, Apr 12",CHW,@,MIN,L,0,4,9.0,4-8,3.0,...,N,15474.0,-1.0,None,Twins,Minnesota Twins,American,Central,#6B8E23,MIN
1,"Sunday, Apr 15",CHW,@,MIN,None,NaN,NaN,NaN,None,NaN,...,None,NaN,NaN,None,Twins,Minnesota Twins,American,Central,#6B8E23,MIN


In [107]:
sox = schedule_and_record(2018, 'CHW')
soxsort = pd.DataFrame ( sox.loc[ ( sox["W/L"].notnull() ) ] )
soxsort.sort_index(ascending=False,inplace=True)
soxlast = soxsort[:1]
soxnext = pd.DataFrame ( sox.loc[ ( sox["W/L"].isnull() ) ] )
soxnext = soxnext[:1]
soxlast = soxnext.append(soxlast)
soxlast = soxlast.rename(columns = {'Tm':'teamID'})
left = soxlast
right = bballJoin
soxagg = pd.merge(left, right, how='left', left_on='Opp', right_on='tres', suffixes=('_x', '_y'))
soxagg = soxagg.sort_values(by='R', ascending=False).reset_index(drop=True)
soxagg

,Date,teamID,Home_Away,Opp,W/L,R,RA,Inn,W-L,Rank,...,D/N,Attendance,Streak,Orig. Scheduled,Team,Tm,League,Division,color,tres
0,"Thursday, Apr 12",CHW,@,MIN,L,0.0,4.0,9.0,4-8,3.0,...,N,15474.0,-1.0,None,Twins,Minnesota Twins,American,Central,#6B8E23,MIN
1,"Sunday, Apr 15",CHW,@,MIN,None,NaN,NaN,NaN,None,NaN,...,None,NaN,NaN,None,Twins,Minnesota Twins,American,Central,#6B8E23,MIN


In [109]:
# get df of teams
soxteams = []
for name,grouped in soxsort.groupby(['Opp']):
    dlist = list([name])
    soxteams.append(dlist)
df = pd.DataFrame(data = soxteams, columns=['Team'])
df

,Team
0,DET
1,KCR
2,MIN
3,TBR
4,TOR


In [78]:
# now aggregate results

# create empty list
soxlist = []
for team in df['Team']:
    # Link team in df to soxteams list in teamlist
    opp = sox['Opp'] == team
    df1 = sox[opp]
    
    # get wins and losses
    w = df1['W/L'] == 'W'
    wins = len( df1[w] )
    l = df1['W/L'] == 'L'
    loses = len( df1[l] )
    
    # average runs/against
    rAvg = np.round( df1['R'].mean() ,2)
    raAvg = np.round( df1['RA'].mean() ,2)
    
    slist = list([team,wins,loses,rAvg,raAvg])
    soxlist.append(slist)
soxagainst = pd.DataFrame(data = soxlist, columns=['teamID','Wins','Loses','AvgRuns','AvgRunsAg'])
soxagainst

,teamID,Wins,Loses,AvgRuns,AvgRunsAg
0,DET,0,3,2.67,5.33
1,KCR,2,0,9.00,5.00
2,MIN,0,1,0.00,4.00
3,TBR,1,2,3.67,4.00
4,TOR,1,2,3.67,7.00


In [79]:
left = soxagainst
right = bballJoin
soxagg = pd.merge(left, right, how='left', left_on='teamID', right_on='tres', suffixes=('_x', '_y'))
soxagg

,teamID,Wins,Loses,AvgRuns,AvgRunsAg,Team,Tm,League,Division,color,tres
0,DET,0,3,2.67,5.33,Tigers,Detroit Tigers,American,Central,#6B8E23,DET
1,KCR,2,0,9.00,5.00,Royals,Kansas City Royals,American,Central,#6B8E23,KCR
2,MIN,0,1,0.00,4.00,Twins,Minnesota Twins,American,Central,#6B8E23,MIN
3,TBR,1,2,3.67,4.00,Rays,Tampa Bay Rays,American,East,#6B8E23,TBR
4,TOR,1,2,3.67,7.00,Blue Jays,Toronto Blue Jays,American,East,#6B8E23,TOR


In [9]:
sox.dtypes

Date                object
Tm                  object
Home_Away           object
Opp                 object
W/L                 object
R                  float64
RA                 float64
Inn                float64
W-L                 object
Rank               float64
GB                  object
Win                 object
Loss                object
Save                object
Time                object
D/N                 object
Attendance         float64
Streak             float64
Orig. Scheduled     object
dtype: object

In [10]:
sox.shape

(12, 19)

In [22]:
sox[:1]
#soxlast=pd.DataFrame( sox.iloc[0] )
#soxlast

,Date,Tm,Home_Away,Opp,W/L,R,RA,Inn,W-L,Rank,GB,Win,Loss,Save,Time,D/N,Attendance,Streak,Orig. Scheduled
12,"Thursday, Apr 12",CHW,@,MIN,L,0.0,4.0,9.0,4-8,3.0,3.5,Berrios,Giolito,None,2:55,N,15474.0,-1.0,None
